# Train deep model + own emmbedings

In [1]:
import re
import pandas as pd
from collections import Counter
from ast import literal_eval
from sklearn.model_selection import train_test_split

liwcPath = '../data/LIWIC/LIWC2007_English100131.dic'

In [2]:
df = pd.read_csv('../data/pandas_data_frame.csv', index_col=0)
all_data = df.where((pd.notnull(df)), '')
all_data['hashtag'] = all_data['hashtag'].apply(literal_eval)

full_text = all_data['tidy_tweet'][(all_data['label']==1.0) | (all_data['label']==0.0)]
y = all_data['label'][(all_data['label']==1.0) | (all_data['label']==0.0)]

In [3]:
full_text[0:10]

0    when father dysfunct selfish drag kid into dys...
1       thank  credit caus they offer wheelchair van  
2                                  bihday your majesti
3                                  love take with time
4                                   factsguid societi 
5    huge fare talk befor they leav chao disput whe...
6                                  camp tomorrow danni
7    next school year year exam think about that   ...
8                                 love land     cavali
9                                          welcom here
Name: tidy_tweet, dtype: object

In [4]:
import sys
sys.path.append("../")
from personal_library.NPL.liwc import LIWCProcessor

liwic = LIWCProcessor(liwcPath)
liwic_text = liwic.corpus2liwc(full_text)
x_train, x_val, y_train, y_val = train_test_split(liwic_text, y, random_state=1992, test_size=0.2)

print(x_train.shape,y_train.shape)
print(x_val.shape,y_val.shape)

(25569, 64) (25569,)
(6393, 64) (6393,)


In [8]:
from personal_library.sce_keras.loss_functions import f1_loss
from personal_library.sce_keras.metrics_functions import f1

from keras.layers import Dense, LSTM, SpatialDropout1D, Dropout
from keras.models import Sequential
from keras.optimizers import Adam

shape = liwic_text.shape
epochs = 40
batch_size = 128
learnRate = 0.01

adam = Adam(lr=learnRate, beta_1=0.9, beta_2=0.999,
            epsilon=None, decay=1e-6, amsgrad=False)

model = Sequential()
model.add(Dropout(0.1,  input_shape=(64,)))
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1,activation='sigmoid'))
model.summary()

model.compile(loss=f1_loss,
              optimizer=adam,
              metrics = ['accuracy', f1])

history = model.fit(x_train, y_train, 
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=2,
                    validation_data=(x_val, y_val))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 16)                528       
_________________________________________________________________
dropout_4 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 17        
Total params: 2,625
Trainable params: 2,625
Non-trainable params: 0
_________________________________________________________________
Train on